# Dubito 

note: 
- nel deck uso un np.array per tracciare il deck, dalle altre parti uso liste
- per ora è uso locale, numeri che si ripetono (senza specificare i semi) e 
- metti un numero di jolly variabile: devi poter decidere quanti deck usare di carte francesi e quanti jolly

### Todoes:
- non ho implementato bene il controllo sul doubt
- implementa il jolly nel controllo sul doubt
- metti che se tutti passano allora viene svuotata la pool e si ricomincia da 0 un giro

### Imports

In [158]:
import numpy as np
import random as random
import re
import time

---
## Deck class

In [159]:
class Cards:
    def __init__(self, num_players):
        self.num_players = num_players
        self.to_draw = 0 
        self.deck = []
        self.pool = [] 
        
        self.create_deck()
    
    def create_deck(self):
        """ Crea un deck con 4 copie di ogni carta (una per seme)
            e 4 jolly, indicati per convenzione col numero 100 """
        
        self.deck = np.array([i for i in range(1,14)]*4 +[100]*4, dtype=int)
        if self.num_players > 5:
            self.deck = np.array[list(self.deck) + list(self.deck)]
        
        self.to_draw = len(self.deck)//self.num_players
        self.shuffle()
        
    def shuffle(self):
        random.shuffle(self.deck)
        
    def add_to_pool(self, cards):
        self.pool += list(cards)
        
    def reset_pool(self):
        self.pool = []
        
                
    def draw(self):
        """ draw an equal amount of cards from the deck for each player.
            Return the cards extracted for one player, while removing
            them from the Deck.
            Note: requires that the Deck remains shuffled"""
        
        # safety check
        assert len(self.deck) >= self.to_draw, "There are not enough cards to draw!"
        
        # extracted cards
        #extracted = np.random.choice(self.deck, self.to_draw, replace = False)
        extracted = self.deck[:self.to_draw]
        
        # remove them from deck
        #self.deck = np.array([i for i in self.deck if i not in extracted])
        self.deck = self.deck[self.to_draw:]
        
        return extracted
    
    
    
def safety_checks_Deck():
    """ Some (not thourough) checks that cards are correctly
        distributed among players, and that the pool gets correctly
        expanded and reset """
    
    c = Cards(3)

    hand1 = c.draw()
    hand2 = c.draw()
    hand3 = c.draw()
    assert len(hand1) == len(hand2) == len(hand3)

    len_hand = len(hand1)
    c.add_to_pool(hand1)
    assert len(c.pool) == len_hand, "Error in adding cards to the pool"

    c.reset_pool()
    assert not c.pool
    
    return

---
## Player class

In [70]:
class Player:
    def __init__(self, deck, id_):
        self.hand = []
        self.deck = deck
        self.id_ = id_
        
    def add_to_hand(self, cards):       
        # volendo puoi sovrascrivere __iadd__ per poter operare comodamente con np arrays
        self.hand += list(cards)
    
    def show_hand(self):
        print("Le tue carte sono: ", sorted(self.hand))
    
    def reset_hand(self):
        self.hand = []
        
    def play_cards(self, cards):
        """ Input: cards -> list.
            Purpose: remove from hand the cards played, 
            and add them to the pool. 
            Controls that each card played is actually in hand,
            otherwise disregards them"""
        
        # remove each played card from the hand, if present, and keep in cards the cards actually played
        cards = [(self.hand.remove(num),num)[1] for _i, num in enumerate(cards) if num in self.hand ]
        add_to_pool = self.deck.add_to_pool(cards)
        
        return cards
    
    
    
def safety_checks_Player():
    """ Checks that, when invalid cards are passed as argument to
        'play_cards', no card actually gets played.
        Checks that all cards correctly get played """
    c = Cards(2)
    p1 = Player(c, 0)
    p2 = Player(c, 1)

    p1.add_to_hand(c.draw())
    p2.add_to_hand(c.draw())

    len_p1_hand = len(p1.hand)
    len_p2_hand = len(p2.hand)
    assert len_p1_hand == len_p1_hand, "Errore in fase di pesca"

    p1.play_cards([22222, 23313, 34414])
    assert len_p1_hand == len(p1.hand), "Some cards were played despite none should have been played"

    p1.play_cards([i for i in p1.hand])
    assert len(p1.hand) == 0, "Non sono state giocate tutte le carte che avrebbero dovuto essere giocate"

    p2.play_cards([i for i in p2.hand][:1] + [32313231])
    assert len(p2.hand) == len_p2_hand-1, "Sono state sbagliate le carte giocate"

---
## Game class

In [187]:
class Game:
    def __init__(self, num_players):
        self.Deck = Cards(num_players)
        self.num_players = num_players
        self.players = [Player(self.Deck, id_=i) for i in range(1, num_players+1)] 
               
        self.turn = 0       
        self.player = 0
        
        self.need_to_declare = True
        
        self.decl_istr  = "Dichiara il tipo di carta che andrà giocata (il numero, non il seme): "
        self.playc_istr = "Inserire numero e seme per ogni carta che vuoi buttare, separando ogni carta con uno spazio: "
        self.doubt_istr = "Insert the number of the player who wants to doubt, if any. Otherwise just press enter: "
        
        self.play()
     
    def first_draw(self):
        for id_ in range(self.num_players):
            player = self.players[id_]
            player.add_to_hand(self.Deck.draw())
        
    def advance_turn(self):
        self.turn += 1
        if self.turn >= self.num_players:
            self.turn = 0
    
    def take_input(self, istrz, min_=0, max_=13, type_=int, none = False, no_self_doubt = False):
        """ Take an input, while controlling that it is
            within a numerical range or that it is of the
            right type, if the relative arguments are specified """
        
        while True:
            ui = input(istrz)
            
# se None è una risposta ammissibile e viene data, restituisci None
            if none == True and not ui:
                return ui
            
# here start checks about type, and min < ui < max, and self doubting
            if type_ is not None:
                try:
                    ui = type_(ui)
                except ValueError:
                    print("Input type must be {0}.".format(type_.__name__))
                    continue
            
            if no_self_doubt == True and ui == self.player.id_:
                print("One can't doubt his own play! :/")
                
            if max_ is not None and ui > max_:
                print("Input must be less than or equal to {0}.".format(max_))
            elif min_ is not None and ui < min_:
                print("Input must be greater than or equal to {0}.".format(min_))
            else:
                return ui
    
    
    def play_cards(self):
        """ ask the user for the cards he wants to play,
            take only those which he actually has in hand,
            use the Player class method to play the cards """
        
# request input cards to play from player x 
        self.player.show_hand()    
        # take input, split based on comma or whitespace, clean resulting empty strings
        #in the thus formed list (e.g. if one uses two whitespaces), convert to int, 
        cards_played = [int(carta) for carta in \
                         list(filter(None, re.split(" |,", input(self.playc_istr))))]
        #cards_played = list(filter(lambda x:x in \
        #                            cards_to_play, \
        #                            self.player.hand))
        
# actually play the cards
        cards_played = self.player.play_cards(cards_played)

# informative prints
        if cards_played:
            print(f"player {self.played.id_} has played {len(cards_played)} {self.card_declared}")
        else:
            print(f"player {self.played.id_} has passed its turn")  
        
        return cards_played
         
        
    def doubt(self, cards_played, who_doubted):
        
        # either way, things will get reset
        self.need_to_declare = True
        
# if the player was honest, reset and give the pool to the doubter
        if cards_played == self.card_declared:
            print("E invece il giocatore {} aveva effettivamente giocato {}{}!" \
                  .format(self.player.id_, len(cards_played), self.card_declared))
            self.players[who_doubted].add_to_hand(self.Cards.pool)
            self.Deck.reset_pool()
            return False

# if the player got correctly doubted, give him the first turn and the bluffer takes the cards
        else:
            print(f"Il giocatore {who_doubted} ha correttamente dubitato del giocatore {self.player.id_}!")
            self.player.add_to_hand(self.Deck.pool)
            self.Deck.reset_pool()
            return True

    def play(self):
        
        self.first_draw()
                
        while True:
            
            """---------------- define player ----------------"""
            
            self.player = self.players[self.turn]
            print("\n----------------------------------------")
            print(f"è il turno del giocatore {self.player.id_}")
            
            
            """---------------- play card ----------------"""
            
            if self.need_to_declare:
                self.player.show_hand()
                # declare the card which will need to be played
                self.card_declared = self.take_input(istrz=self.decl_istr)
                self.need_to_declare = False
                
            print(f"La carta da giocare è il {self.card_declared}")
            
            # play cards
            cards_played = self.play_cards()         
            
            """---------------- doubt ----------------"""
            
            who_doubted = None
            # se non è stato passato il turno, apri la fase di dubbio
            if cards_played:
                # if someone wants to doubt, insert his player number in this input
                who_doubted = self.take_input(self.doubt_istr, 
                                              min_= 0,
                                              max_=self.num_players,
                                              type_=int,
                                              none = True,
                                              no_self_doubt = True)
                if who_doubted:
                    correctness_doubt = self.doubt(cards_played, who_doubted)   
                    time.sleep(3)
            
            """---------------- check victory ----------------"""
            
            if not self.player.hand:
                print(f"\n Player {self.player._id} has won the game, ggs! \n")
                return
            
            """---------------- manage turns ----------------"""
            
            # if there was a doubt, give the turn to the winner of the outcome
            if who_doubted and correctness_doubt == True:
                self.turn = who_doubted
                
            # else regularly advance the turn
            elif not who_doubted:
                self.advance_turn()
        
        return

In [190]:
# Let's run some safety checks before instantiating the game!
safety_checks_Deck()
safety_checks_Player()

# Let's instantiate the game!
num_players = 2
g = Game(num_players)


----------------------------------------
è il turno del giocatore 1
Le tue carte sono:  [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8, 10, 10, 10, 12, 12, 13, 13, 13, 13, 100]


KeyboardInterrupt: Interrupted by user

---